In [25]:
import sys
import math
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedShuffleSplit
from MLPClass import MLPClass

# Defining Functions...

In [6]:
def Preprocessing(data):
    label_target = data[0]
    target = []
    
    for i in range(len(label_target)):
        if label_target[i] == 1:
            element = [1, 0, 0]
        elif label_target[i] == 2:
            element = [0, 1, 0]
        elif label_target[i] == 3:
            element = [0, 0, 1]
        target.append(element)
    
    x_without_target = data.drop([0], axis = 1)
    temp             = x_without_target.values.astype(float)
    min_max_scaler   = preprocessing.MinMaxScaler()
    x_scaled         = min_max_scaler.fit_transform(temp)
    predictive       = pd.DataFrame(x_scaled)
    
    return np.array(predictive), np.array(target), (label_target)

In [7]:
def Centers(predictive, label_target):
    old_value = label_target[0]
    old_idx = 0
    centers_dataset = []
    groups = []
    
    for idx, value in label_target.items():
        
        if old_value != value or idx == len(label_target) - 1:
            new_df = predictive.iloc[old_idx:idx]
            centers_dataset.append(new_df.mean(axis = 0))
            old_idx   = idx
            old_value = value
            groups.append(new_df)
            
    centers_dataset = np.array(centers_dataset)
    return centers_dataset, groups

In [21]:
def getMiniBatches(x, y, size): 
    mini_batches  = []    
    try:
        aux = y.shape[1]        
    except:
        aux = 1        

    data          = np.hstack((x, np.array(y).reshape(y.shape[0], aux)))    
    n_minibatches = data.shape[0] // size 
    i             = 0
    
    np.random.shuffle(data) # Random data
    for i in range(n_minibatches + 1):
        mb     = np.array(data[i * size:(i + 1)*size, :])         
        x_mini = mb[:, :-aux]
        y_mini = mb[:, x.shape[1]:].reshape((-aux, aux))    
        mini_batches.append((x_mini, y_mini)) 

    if data.shape[0] % size != 0: # If there needs to create one adittional mini batch
        mb     = data[i * size:data.shape[0]] 
        x_mini = mb[:, :-aux] 
        y_mini = mb[:, x.shape[1]:].reshape((-aux, aux))
        mini_batches.append((x_mini, y_mini))

    return np.array(mini_batches)

In [8]:
# vj = ||x - cj|| -----------  v = [v1,...,vj]
# x: vetor de entrada
# c: centro da função radial  (miu)
# s: largura da função radial (sigma)

# Others functions:
def closest(x,vectors):
    distance = []
    for i in range(len(vectors)):
        distance.append(np.linalg.norm(np.array(x)-np.array(vectors[i])))
    index = distance.index(min(distance))
    return vectors[index]

def flatten(Matrix, axis):
    F1 = Matrix[0]
    for i in range(len(Matrix)-1):
        F2 = Matrix[i+1]
        if (axis==1):
            Flat = np.concatenate((F1, F2), axis=1)
        else:
            Flat = np.concatenate((F1, F2), axis=0)
        F1 = Flat        
    return Flat

# Definition of widths:
# Heuristic 1: s = 1/m * somatorio(i=1,m)||ci-c|| -----> c centro mas cercano a ci 
def heuristic_1(centers):
    sj = []
    for j in range(len(centers)):
            x = centers[0]
            centers.remove(x)
            y = closest(x, centers)  # centro mas cercano al centro actual x
            sj.append(np.linalg.norm(np.array(x)-np.array(y)))
            centers.append(x)
    return (np.ones([1,3])*(float(1/len(centers))*np.sum(sj)))      

# Heuristic 2: si = alpha * ||ci-c||   -----> 1<alpha<1.5 c centro mas cercano al centro ci 
def heuristic_2(alpha,centers):
    s = []
    for i in range(len(centers)):
        x = centers[0]
        centers.remove(x)
        y = closest(x, centers)  # centro mas cercano al centro actual x
        s.append(alpha*np.linalg.norm(np.array(x)-np.array(y)))
        centers.append(x)
    return np.array(s)

# heuristic 3: si = distância média de seu centro aos N vetores de entrada mais próximos
def heuristic_3(N,centers,groups):
    # N<=47
    s = []
    for i in range(len(centers)):
        center_current = centers[i]
        group_current = np.array(groups[i])
        sj = []
        for j in range(N):
            sj.append(np.linalg.norm(group_current[j]-np.array(center_current)))
        s.append(float(1/N)*np.sum(sj))
    return s

# heuristic 4:  # si = valor constante (geralmente 1)
def heuristic_4(): 
    return(np.ones([1,3]))

def sigma_heuristics(centers, groups, N, alpha, option = 1):
    if option==1:
        sigma = heuristic_1(centers)
    elif option==2:
        sigma = heuristic_2(alpha, centers)
    elif option==3:
        sigma = heuristic_3(N, centers, groups)
    else:
        sigma = heuristic_4()
    return sigma 
        
# Activations functions:
def sigmoid(x):
    return (1 /(1+ np.exp(-x)))

# Radial functions:
def Gaussian(v,sigma): 
    # phi(v) = exp(-(v^2)/ 2*sigma^2)
    return (np.exp(-(v**2)/ 2*(sigma**2)))

def Multiquadratic(v,sigma):
    # phi(v) = raiz(v^2 + sigma^2)
    return (np.sqrt(v**2 + sigma**2))

def Thin_Plate_Spline(v):
    # phi(v) = v^2 * log(v)
    return (v**2 * np.log(v))

def phi(x, center, sigma, function):
    v = np.linalg.norm((np.subtract(x,center)))
    if function =='Gaussian':
        phi = Gaussian(v,sigma)
    elif function =='Multiquadratic':
        phi = Multiquadratic(v,sigma)
    elif function =='Thin_Plate_Spline':
        phi = Thin_Plate_Spline(v)
    return phi

In [9]:
def split_dataset(data, target, old_target = []):
    stratSplit = StratifiedShuffleSplit(n_splits=1, test_size=0.8, random_state=42)
    for test_idx, train_idx in stratSplit.split(data, target):
        data_train   = data[train_idx]
        target_train = target[train_idx]

        data_test = data[test_idx]
        target_test = target[test_idx]
        
        old_target_test = old_target[test_idx]
        old_target_train = old_target[train_idx]
        return np.array(data_train), np.array(data_test), np.array(target_train), np.array(target_test), np.array(old_target_test), np.array(old_target_train)

In [10]:
def forward_RBF(Input, centers, W, number_neurons, sigma, function):
    phi_values = []
    for i in range(len(centers)):
        phi_values.append(phi(Input, centers[i], sigma[0][i], function))
    phi_values.append(1)# Adding bias
    phi_values = np.array(phi_values)
    output = sigmoid(np.dot(phi_values, np.transpose(np.matrix(W)))) 
    return output, phi_values    

In [ ]:
# def heuristic_1(centers, groups):
#     sj     = []
#     for i in range(len(groups)):
#         group_current = groups[0]
#         center_current = centers[0]
#         groups.remove(group_current)
#         centers.remove(center_current)
#         Groups = []
#         for k in range(len(groups)):
#             Groups.append(np.array(groups[k]))
#         Groups = flatten(Groups, axis = 0) 
#         for j in range(len(group_current)):
#             point_closest = closest(group_current[j], Groups)
#             if (point_closest in np.array(groups[0])):
#                 sj.append(np.linalg.norm(np.array(center_current)-np.array(centers[0])))
#             elif (point_closest in groups[1]):
#                 sj.append(np.linalg.norm(np.array(center_current)-np.array(centers[1])))       
#         groups.append(group_current)  
#         centers.append(center_current)
#     return (np.ones([1,3])*(float(1/177)*np.sum(sj)))

# Loading Data...

In [12]:
data = pd.read_csv("wine.data", header = None)

# Pre-Processing Data...

In [20]:
predictive, target, label_target = Preprocessing(data)
centers_dataset, groups = Centers(pd.DataFrame(predictive), label_target)
data_train, data_test, target_train, target_test, old_target_test, old_target_train = split_dataset(predictive, target, label_target)

# Defining General Parameters...

In [28]:
sizeMiniBatch = 1
lowWeight     = -1
highWeight    = 1
learningRate  = 0.002
alphaMomentum = 0
maxNumberOfIterations = 115

# Testing MLP...

In [29]:
mini_batches  = getMiniBatches(data_train, target_train, sizeMiniBatch)
size_x        = len(mini_batches[0][0][0])
size_y        = 3
w_middle      = []
w_output      = []
wb_middle     = []
wb_output     = []


neuronsOnHiddenLayer = [3]
MLPClassifier        = MLPClass(lowWeight, highWeight, neuronsOnHiddenLayer, size_x, size_y)

for epoch in range(maxNumberOfIterations):
    nb = 0
    for mb in mini_batches:
        # print('Numero Batch: ', nb)
        if len(mb[0]) != 0:	# Checks that mini batch has data
            x_mini = mb[0]
            y_mini = mb[1]

            w_middle, w_output, wb_middle, wb_output = MLPClassifier.fit_by_mini_batch(x_mini, y_mini, learningRate, alphaMomentum)
        nb = nb + 1
    errorGlobal = 0
    y_pred = []
    for i in range(len(data_test)):
        y_layer, y_net, error = MLPClassifier.forward(data_test[i], target_test[i], w_middle, w_output, wb_middle, wb_output)
        result = np.where(y_net == np.amax(y_net))
        y_pred.append(result[0][0] + 1)
        errorGlobal = errorGlobal + error

    errorGlobal = errorGlobal / len(data_test)
    if (epoch + 1) % 50 == 0 or epoch == maxNumberOfIterations - 1:
        print('Accuracy on iteration ' + str(epoch + 1) + ': ' + str(accuracy_score(np.array(old_target_test),np.array(y_pred))))
                

Accuracy on iteration 50: 0.4
Accuracy on iteration 100: 0.4
Accuracy on iteration 115: 0.42857142857142855


# Testing RBF...

In [19]:
centers = centers_dataset
number_neurons = 3
W        = np.random.uniform(low = -1, high = 1, size = [number_neurons, len(centers)+1])
function = 'Gaussian'
N = 40 # N<=47
alpha = 1
sigma = sigma_heuristics(list(centers), groups, N, alpha, option = 1)


for n in range(maxNumberOfIterations):
    for i in range(len(data_train)):
        output, phi_values = forward_RBF(data_train[i], centers, W, number_neurons, sigma, function)
        aux = learning_rate * np.multiply(np.transpose(np.matrix(target_train[i] - output)), np.array(phi_values))
        W = W + aux
    
    if (n + 1) %50 == 0 or n == maxNumberOfIterations -1:
        y_pred = []
        for i in range(len(data_test)):
            output, phi_values = forward_RBF(data_test[i], centers, W, number_neurons, sigma, function)
            result = np.where(output == np.amax(output))
            a = np.argmax(output)
            y_pred.append(a + 1)
        print('Accuracy on iteration ' + str(n+1) +': '+ str(accuracy_score(np.array(old_target_test),np.array(y_pred))))

        
# print('W: ', W)   

Accuracy on iteration 50: 0.4
Accuracy on iteration 100: 0.6571428571428571
Accuracy on iteration 115: 0.7714285714285715
